In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,171 kB]
Get:11 http://archive.ubuntu.c

In [ ]:
import csv
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
# Function to scrape listing elements from the Swiggy website
def scrape_listings(soup):
    return soup.select('a.RestaurantList__RestaurantAnchor-sc-1d3nl43-3')

# Function to scrape the restaurant name from a listing
def scrape_restaurant_name(listing):
    name_element = listing.select_one('div.sc-beySbM.iKLEMo')
    return name_element.text.strip() if name_element else None

# Function to scrape the image URL from a listing
def scrape_image_url(listing):
    img_element = listing.select_one('div.sc-jsEegt img')
    return img_element['src'] if img_element else None

# Function to scrape the rating from a listing
def scrape_rating(listing):
    rating_element = listing.select_one('span.sc-beySbM.jAtOQO')
    return rating_element.text.strip().split('•')[0].strip() if rating_element else None

# Function to scrape the cuisine type from a listing
def scrape_cuisine(listing):
    cuisine_element = listing.select('div.sw-restaurant-card-descriptions-container div.sc-beySbM.bRTXBF')
    return cuisine_element[0].text.strip() if cuisine_element else None

# Function to scrape the location from a listing
def scrape_location(listing):
    location_element = listing.select('div.sw-restaurant-card-descriptions-container div.sc-beySbM.bRTXBF')
    return location_element[1].text.strip() if len(location_element) > 1 else None

# Function to scrape the delivery time from a listing
def scrape_delivery_time(listing):
    delivery_time_element = listing.select_one('div.sc-beySbM.brneVe')
    if delivery_time_element:
        split_text = delivery_time_element.text.strip().split('•')
        if len(split_text) > 1:
            return split_text[1].strip()
        else:
            return split_text[0].strip()  # Return the first element if no separator is found
    return None

In [ ]:
!apt-get update
!apt install firefox-geckodriver
!pip install selenium webdriver_manager

import csv
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set up Firefox options
firefox_options = FirefoxOptions()
firefox_options.add_argument("--headless")

# Define your scraping functions here (scrape_listings, scrape_restaurant_name, etc.)

def main():
    # Set up Selenium WebDriver for Firefox
    driver = webdriver.Firefox(options=firefox_options)

    # Navigate to the Swiggy website URL
    url = 'https://www.swiggy.com/city/bangalore'
    driver.get(url)

    # Open a CSV file to write the restaurant data
    with open('swiggy_data.csv', 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['restaurant_name', 'image_url', 'rating', 'cuisine', 'location', 'delivery_time']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()

        while True:
            # Wait for the page to load
            time.sleep(5)

            # Get the page source and create a BeautifulSoup object
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Scrape restaurant listings
            listings = scrape_listings(soup)

            # Iterate through each listing and extract restaurant information
            for listing in listings:
                restaurant_name = scrape_restaurant_name(listing)
                image_url = scrape_image_url(listing)
                rating = scrape_rating(listing)
                cuisine = scrape_cuisine(listing)
                location = scrape_location(listing)
                delivery_time = scrape_delivery_time(listing)

                # Write the restaurant information to the CSV file
                writer.writerow({
                    'restaurant_name': restaurant_name,
                    'image_url': image_url,
                    'rating': rating,
                    'cuisine': cuisine,
                    'location': location,
                    'delivery_time': delivery_time
                })

            # Check if there's a "Show More" button
            try:
                show_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "div.RestaurantList__ShowMoreContainer-sc-1d3nl43-0"))
                )
                driver.execute_script("arguments[0].click();", show_more_button)
            except:
                # If there's no "Show More" button, we've reached the end
                break

    # Close the browser
    driver.quit()

    print("Data extraction complete. Check the 'swiggy_data.csv' file.")

# Run the main function
if __name__ == "__main__":
    main()

# Download the CSV file
from google.colab import files
files.download('swiggy_data.csv')


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>